<a href="https://colab.research.google.com/github/arshad831/1-Langchain/blob/main/crewai_multiagent_2025-16oct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.19.2
    Uninstalling typer-0.19.2:
      Successfully uninstalled typer-0.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.0 requires typer<1.0,>=0.12, but you have typer 0.9.4 which is incompatible.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano-2025-04-14'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [2]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [ ]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "218873db9b73d345d3083e2f1f29cb74b14d4ddb" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "How close are we to Artificial General Intelligence ?"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on How close are we to Artificial General Intelligence ?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news on How close are we to Artificial General Intelligence in order to prioritize them appropriately.

Action: Search the internet
Action Input: {"search_query": "latest trends in Artificial General Intelligence"} 


Search results: Title: 6 AI trends you'll see more of in 2025 - Microsoft Source
Link: https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/
Snippet: In 2025, AI will evolve from a tool for work and hom

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# How Close Are We to Achieving Artificial General Intelligence?

## Introduction
Artificial General Intelligence (AGI) has long been a goal for scientists and tech enthusiasts around the world. The concept of creating a machine that can perform any intellectual task that a human can do has sparked endless debates and discussions. In this article, we will delve into the latest trends, key players, and noteworthy news in the field of AGI. We will also explore the target audience for AGI content and the implications of its development on society and ethics.

## Latest Trends in Artificial General Intelligence
The evolution of AI from a mere tool to an essential part of our daily lives has been remarkable. AI-powered agents are becoming increasingly autonomous, simplifying tasks and improving efficiency in various industries. Breakthroughs in AI technology are shaping the future of AGI, bringing us closer to achieving this elusive goal.

Key players in the field of AGI, such as Apple, Microsoft, NVIDIA, Alphabet, and leading researchers like Geoff Hinton, Yann LeCun, and Yoshua Bengio, are pushing the boundaries of what is possible in artificial intelligence. Their contributions to the development of AGI are paving the way for a future where intelligent machines can think, learn, and adapt like humans.

## Noteworthy News on Artificial General Intelligence
Recent developments in the world of AGI have been nothing short of groundbreaking. The SoftBank chief's prediction of achieving Artificial Superintelligence (ASI) within the next decade has raised eyebrows and sparked discussions about the implications of such a technological advancement. In Albania, an AI minister is aiming to use AI to eliminate corruption, showcasing the potential of AGI to bring about positive societal change.

The implications of AGI development on society and ethics cannot be understated. As we move closer to achieving Artificial General Intelligence, questions about job displacement, privacy concerns, and ethical considerations are becoming more prevalent. It is crucial for us to address these issues proactively and ensure that the benefits of AGI outweigh the potential risks.

## Target Audience for AGI Content
Marketers, researchers, tech enthusiasts, and AI professionals are among the primary target audience for AGI content. Business owners looking to leverage AI for growth and efficiency can benefit from staying informed about the latest trends and developments in the field. The general audience seeking insights into the future of technology will also find value in learning about the potential of AGI and its impact on society.

As we navigate the complex landscape of Artificial General Intelligence, it is essential to stay informed and engaged with the latest developments. Join us on this journey towards achieving AGI and discover the endless possibilities it holds for the future.

In conclusion, the road to achieving Artificial General Intelligence is paved with challenges and opportunities. With the contributions of top players in the field, the latest trends in AI technology, and the dedication of researchers and tech enthusiasts worldwide, we are closer than ever to realizing the dream of creating machines that can think and learn like humans. As we continue on this journey, it is crucial to consider the ethical implications and societal impact of AGI development to ensure a future where intelligent machines coexist harmoniously with humanity.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Chatgpt Agent by openAI"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Chatgpt Agent by openAI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news on Chatgpt Agent by openAI to create an engaging and informative blog article.

Action: Search the internet
Action Input: {"search_query": "latest trends Chatgpt Agent by openAI"} 


Search results: Title: Introducing ChatGPT agent: bridging research and action - OpenAI
Link: https://openai.com/index/introducing-chatgpt-agent/
Snippet: ChatGPT now thinks and acts, proactively choosing from a toolbox of agentic skills to complete tasks for you using its own computer.
---
T

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of ChatGPT Agent by OpenAI: A Comprehensive Guide

ChatGPT Agent by OpenAI has made a significant impact on the artificial intelligence landscape, introducing cutting-edge features and capabilities that have captured the attention of tech companies, researchers, developers, and organizations of all sizes. With its ability to control a computer, recent rollout for $20 Plus users, and advanced functionalities, ChatGPT Agent is paving the way for the future of AI technology.

The launch of ChatGPT Agent has sparked excitement in the tech industry, showcasing new features and enhancements that solidify its position as a leading AI technology. The team behind OpenAI's ChatGPT Agent has demonstrated expertise and dedication to innovation, pushing the boundaries of what AI can achieve. News of its ability to control a computer has particularly resonated with businesses seeking to streamline processes and increase productivity.

ChatGPT Agent caters to a diverse audience, including tech companies, researchers, developers, and organizations of all sizes. Businesses looking to enhance workflows and productivity can leverage ChatGPT Agent effectively. Furthermore, individuals interested in AI technology and its applications will find ChatGPT Agent to be a valuable tool for exploring the possibilities of artificial intelligence.

ChatGPT Agent offers a wide array of features that differentiate it from other AI technologies. Its advanced AI agent capabilities can assist users in various tasks, from answering questions to generating creative content. The virtual assistant features make it a valuable tool for automation and productivity enhancement, enabling businesses to streamline operations and improve efficiency.

The applications of ChatGPT Agent extend across industries and sectors, providing real-world benefits. Tech companies can enhance customer service with instant support, while researchers can expedite data analysis and research processes. Developers can create interactive experiences by integrating ChatGPT Agent into their applications, showcasing its versatility and adaptability.

While the benefits of using ChatGPT Agent are evident, challenges such as data security and ethical considerations must be addressed when integrating AI technology. It is crucial for businesses to weigh these factors before implementing ChatGPT Agent into their operations. Looking ahead, ChatGPT Agent is poised to become an essential tool as AI technology progresses, offering innovative solutions for businesses and individuals alike.

In conclusion, ChatGPT Agent by OpenAI is a transformative AI technology that is reshaping interactions with machines and task automation. With its advanced capabilities, real-world applications, and potential advantages, ChatGPT Agent is a valuable asset for those in the tech industry, researchers, developers, and anyone interested in AI technology. Embracing the future of AI innovation, ChatGPT Agent leads the way towards a smarter and more efficient future.

Take Action: Explore ChatGPT Agent by OpenAI further and unlock the potential of this advanced AI technology. Businesses seeking enhanced productivity should consider integrating ChatGPT Agent into their operations. For more information on ChatGPT Agent and its capabilities, visit OpenAI's official website and reputable sources.

In [ ]:
Markdown(result)

# Deploy this crew in gradio

In [3]:
pip install gradio

  Using cached typer-0.19.2-py3-none-any.whl.metadata (16 kB)
Using cached typer-0.19.2-py3-none-any.whl (46 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.19.2 which is incompatible.


In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS CrewAI Article Writer with Researcher, Writer, and Editor",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91fe87c3d58ed07138.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
from google.colab import files
import shutil

# Step 1: Create target directory
upload_dir = "blog-posts"
os.makedirs(upload_dir, exist_ok=True)
print(f"✅ Directory '{upload_dir}' is ready.")

✅ Directory 'blog-posts' is ready.


In [ ]:


# Step 2: Upload file(s)
uploaded_files = files.upload()

# Step 3: Move uploaded files to the target folder
for filename in uploaded_files.keys():
    shutil.move(filename, os.path.join(upload_dir, filename))
    print(f"📥 Moved '{filename}' to '{upload_dir}/'")

print("✅ All files uploaded and saved.")


Saving Social Media Guidelines for UAE gov 2021 En.pdf to Social Media Guidelines for UAE gov 2021 En.pdf
📥 Moved 'Social Media Guidelines for UAE gov 2021 En.pdf' to 'blog-posts/'
✅ All files uploaded and saved.


In [ ]:
# Already executed in Colab
docs_tool = DirectoryReadTool(directory='blog-posts')


In [ ]:
from crewai_tools import DirectoryReadTool

# Tool setup (ensure directory exists)
docs_tool = DirectoryReadTool(directory='blog-posts')

# Agent using uploaded documents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover insights using uploaded documents and online sources",
    backstory="You're an expert in using both uploaded files and web search to produce comprehensive insights.",
    tools=[docs_tool],  # ← Enable document access
    allow_delegation=False
)


In [ ]:
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)

result = crew.kickoff(inputs={"topic": "AI Regulation"})
print(result)


In [4]:
import gradio as gr
from crewai import Agent, Task, Crew, Process

# ---------- Agents ----------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for {product_brand}",
    backstory=(
        "A senior analyst skilled in competitor intelligence, audience segmentation, "
        "channel dynamics, and market sizing, with a bias for actionable insights."
    ),
    allow_delegation=False,
    verbose=True
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy for {product_brand}",
    backstory=(
        "A veteran strategist who crafts positioning, messaging pillars, channel mix, "
        "and KPI frameworks—aligning cross-functional stakeholders and timelines."
    ),
    # Manager/orchestrator: can hand off sub-tasks or ask questions to teammates
    allow_delegation=True,
    verbose=True
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling creative concepts and a content calendar",
    backstory=(
        "A concept-to-copy creative who converts strategy into campaign ideas, ad copy, "
        "social posts, and SEO-ready long-form content."
    ),
    allow_delegation=False,
    verbose=True
)

# ---------- Crew runner ----------
def run_crewai_marketing_strategy(product_brand: str, target_audience: str, objective: str):
    # Compose a shared topic string for clarity across tasks
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    # 1) Market Analysis
    market_analysis_task = Task(
        description=(
            "Conduct a concise but thorough market analysis for the topic: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section above, "
            "ending with a 5–8 point summary of the most actionable insights."
        ),
        agent=lead_market_analyst,
        input_variables={"topic": topic}
    )

    # 2) Strategy Formulation (manager)
    strategy_task = Task(
        description=(
            "Using the Market Analysis brief, craft a go-to-market strategy for {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output=(
            "A strategy document with clear sections as listed, plus a one-page executive summary."
        ),
        agent=chief_marketing_strategist,
        input_variables={"topic": topic},
        context=[market_analysis_task]
    )

    # 3) Creative & Content Plan
    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search, paid social), "
            "(c) a 4-week content calendar (blog/LI/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, and intended KPIs, and (d) landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output=(
            "Campaign concepts + copy, a tabular content calendar, and a structured LP outline."
        ),
        agent=creative_content_creator,
        input_variables={"topic": topic},
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task]

    )

    result = crew.kickoff()
    return result

# ---------- Gradio app ----------
def generate_strategy(product_brand, target_audience, objective):
    return run_crewai_marketing_strategy(product_brand, target_audience, objective)

iface = gr.Interface(
    fn=generate_strategy,
    inputs=[
        gr.Textbox(lines=1, label="Product / Brand", placeholder="e.g., SaaS analytics platform"),
        gr.Textbox(lines=1, label="Target Audience", placeholder="e.g., mid-market product managers in EMEA"),
        gr.Textbox(lines=1, label="Primary Objective", placeholder="e.g., drive free trials / demo requests")
    ],
    outputs=gr.Textbox(lines=28, label="Marketing Strategy & Content Plan"),
    title="DDS • AI Crew for Marketing Strategy",
    description="Provide brand, audience, and objective. The crew analyzes the market, builds a GTM strategy, and outputs creative + a content calendar."
)

if __name__ == "__main__":
    iface.launch()


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09c4a3a5e0c8a1efbe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from crewai import Agent, Task, Crew, Process

# --- Agents ---
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,

)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,   # manager must be allowed to delegate
    verbose=True,

)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,

)

def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    # --- Tasks ---
    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    # --- Crew (hierarchical: strategist manages delegation/routing) ---
    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        #process=Process.hierarchical,
        #manager_agent=chief_marketing_strategist,
        verbose=True
    )

    return crew.kickoff()


In [ ]:
brand     = "SaaS analytics platform"
audience  = "Mid-market product managers in EMEA"
objective = "Drive free trials"

result = run_marketing_crew(brand, audience, objective)
print("\n=== OUTPUT ===\n")
print(result)


 [DEBUG]: == Working Agent: Lead Market Analyst
 [INFO]: == Starting Task: Conduct a concise market analysis for: SaaS analytics platform | Audience: Mid-market product managers in EMEA | Objective: Drive free trials. Cover: (1) ICP & segments, (2) JTBD/pain points & objections, (3) competitive landscape & whitespace, (4) demand signals & seasonality, (5) channel dynamics (search/social/email/partners/events), (6) keyword themes & content gaps, (7) risks/assumptions.


> Entering new CrewAgentExecutor chain...
I have the expertise and tools to deliver a comprehensive market analysis for the SaaS analytics platform targeted towards mid-market product managers in EMEA.

Final Answer:

1. ICP & Segments:
- Ideal Customer Profile (ICP): Mid-market product managers in EMEA with a focus on data-driven decision-making.
- Segments: Identify key segments such as industry verticals, company sizes, and pain points.

2. JTBD/Pain Points & Objections:
- Job To Be Done (JTBD): Provide actionable ins

In [5]:
from crewai import Agent, Task, Crew
import gradio as gr
import re
from datetime import datetime
from pathlib import Path

# ----------------------------
# Your existing agents
# ----------------------------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,
    verbose=True,
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,
)

# Optional: a focused social copywriter (only used if toggled)
social_copywriter = Agent(
    role="Social Copywriter",
    goal="Turn strategy into platform-appropriate copy with strong hooks and clear CTAs.",
    backstory=("Skilled at LinkedIn thought-leadership, X brevity, Instagram captions, "
               "and long-form posts that convert."),
    allow_delegation=False,
    verbose=False,
)

# ----------------------------
# Core crew (kept same logic)
# ----------------------------
def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        verbose=True
    )
    return crew.kickoff()  # returns the full text output

# ----------------------------
# Helpers (small + simple)
# ----------------------------
def _first_n_points(text: str, n: int = 5):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    bullets = []
    for l in lines:
        if l.startswith(("-", "*", "•")) or re.match(r"^\d+[\.\)]", l) or len(l) > 50:
            bullets.append(l.lstrip("-*• ").strip())
        if len(bullets) >= n: break
    if not bullets:
        parts = [p.strip() for p in text.split("\n\n") if p.strip()]
        bullets = parts[:n]
    return bullets[:n]

def _hashtags(csv_tags: str) -> str:
    tags = [t.strip().replace("#", "") for t in (csv_tags or "").split(",") if t.strip()]
    return "" if not tags else " " + " ".join(f"#{t}" for t in tags)

def _truncate_chars(s: str, max_chars: int) -> str:
    return s if len(s) <= max_chars else s[:max_chars - 1] + "…"

# ----------------------------
# Templates (no extra LLM call)
# ----------------------------
def tpl_linkedin(strategy, brand, audience, objective, hashtags, max_words=180):
    hook = f"{brand}: a sharper path to {objective} for {audience}."
    pts = "\n".join([f"- {p}" for p in _first_n_points(strategy, 5)])
    body = f"""**{hook}**

**Key insights**
{pts}

**Next 90 days**
- Prove the positioning with fast tests
- Double down on channels with strongest signals
- Track leading KPIs weekly

CTA: Comment “PLAYBOOK” if you want the GTM outline.{_hashtags(hashtags)}
"""
    words = body.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else body

def tpl_tweet(strategy, brand, audience, objective, hashtags, max_chars=270):
    points = _first_n_points(strategy, 3)
    msg = f"{brand} → {objective} for {audience}: " + " | ".join(_truncate_chars(p, 80) for p in points)
    return _truncate_chars(msg + _hashtags(hashtags), max_chars)

def tpl_article(strategy, brand, audience, objective, hashtags, max_words=800):
    intro = (f"{brand} is targeting {audience} to achieve {objective}. "
             "Here’s the distilled market analysis, GTM strategy, and a 90-day plan.")
    text = f"""# {brand}: GTM Playbook for {audience}

## Why this matters
{intro}

## Strategy in brief
- Positioning & messaging pillars
- Priority segments & channel mix
- Offers/CTAs, KPI tree
- 90-day roadmap & budget split

## Research & Strategy Notes
{strategy}

## What to do next
1) Validate 1–2 offers with tight ICP cohorts
2) Launch a 2-channel test with weekly KPI reviews
3) Scale what converts, archive what doesn’t

*Updated: {datetime.utcnow().strftime("%Y-%m-%d")}*{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

def tpl_instagram(strategy, brand, audience, objective, hashtags, max_chars=2200):
    pts = _first_n_points(strategy, 3)
    caption = (f"{brand} | {objective} for {audience}\n"
               + "\n".join([f"• {p}" for p in pts])
               + "\n\nCTA: Save & share with your team." + _hashtags(hashtags))
    return _truncate_chars(caption, max_chars)

def tpl_facebook(strategy, brand, audience, objective, hashtags, max_chars=3000):
    pts = _first_n_points(strategy, 4)
    post = (f"{brand} — how we’ll reach {audience} and drive {objective}:\n"
            + "\n".join([f"- {p}" for p in pts])
            + "\n\nComment if you want the 90-day playbook." + _hashtags(hashtags))
    return _truncate_chars(post, max_chars)

def tpl_youtube_desc(strategy, brand, audience, objective, hashtags, max_chars=5000):
    pts = _first_n_points(strategy, 5)
    desc = (f"{brand} GTM for {audience} | Objective: {objective}\n\nKey Points:\n"
            + "\n".join([f"- {p}" for p in pts])
            + "\n\nChapters:\n00:00 Intro\n00:30 Market Signals\n02:00 Strategy\n04:00 Next Steps\n"
            + "\nCTA: Subscribe for weekly GTM breakdowns." + _hashtags(hashtags))
    return _truncate_chars(desc, max_chars)

def tpl_threads(strategy, brand, audience, objective, hashtags, max_chars=500):
    pts = _first_n_points(strategy, 2)
    post = f"{brand} → {objective} for {audience}\n" + "\n".join([f"• {p}" for p in pts]) + _hashtags(hashtags)
    return _truncate_chars(post, max_chars)

def tpl_medium(strategy, brand, audience, objective, hashtags, max_words=1000):
    # Medium: same as article but lighter header
    text = f"""# {brand} GTM: From Signals to Scale

**Audience:** {audience} | **Objective:** {objective}

**In this post:** positioning, channel mix, KPIs, and a practical 90-day plan.

## Insights & Plan
{strategy}

## Close
If this resonates, highlight what you’d test first and why.{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

# ----------------------------
# Optional LLM copywriter (reuses Crew/Agent)
# ----------------------------
def llm_copywriter(strategy_text, brand, audience, objective, tone, platform,
                   hashtags, li_words, tweet_chars, article_words):
    limits = {
        "LinkedIn": f"≈ {li_words} words",
        "X (Twitter)": f"≤ {tweet_chars} chars",
        "Article": f"≈ {article_words} words",
        "Instagram": "≤ 2200 chars",
        "Facebook": "≤ 3000 chars",
        "YouTube Description": "≤ 5000 chars",
        "Threads": "≤ 500 chars",
        "Medium Article": f"≈ {article_words} words",
    }
    req = f"""Create a {platform} post from the GTM strategy.

Brand: {brand}
Audience: {audience}
Objective: {objective}
Tone: {tone}
Hashtags: {hashtags or '(none)'}
Length limit: {limits.get(platform, 'keep concise')}

Requirements:
- Strong first-line hook
- 1–3 concrete insights from the strategy (no clichés)
- Clear CTA
- Respect platform style and length

--- STRATEGY ---
{strategy_text}
"""
    task = Task(description=req, expected_output=f"{platform} copy ready to publish.", agent=social_copywriter)
    crew = Crew(agents=[social_copywriter], tasks=[task], verbose=False)
    return crew.kickoff()

# ----------------------------
# Gradio glue (simple dropdown)
# ----------------------------
PLATFORMS = [
    "LinkedIn",
    "X (Twitter)",
    "Article",
    "Instagram",
    "Facebook",
    "YouTube Description",
    "Threads",
    "Medium Article",
]

def generate(product_brand, target_audience, objective,
             platform, tone, hashtags, use_llm,
             li_max_words, tweet_max_chars, article_max_words):

    if not product_brand or not target_audience or not objective:
        return "Please fill Brand, Audience, and Objective.", "", None

    # 1) Run your main Crew once
    strategy = run_marketing_crew(product_brand, target_audience, objective)

    # 2) Pick template or LLM route
    if use_llm:
        social = llm_copywriter(strategy, product_brand, target_audience, objective,
                                tone, platform, hashtags,
                                li_max_words, tweet_max_chars, article_max_words)
    else:
        if platform == "LinkedIn":
            social = tpl_linkedin(strategy, product_brand, target_audience, objective, hashtags, li_max_words)
            fname = f"linkedin_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "X (Twitter)":
            social = tpl_tweet(strategy, product_brand, target_audience, objective, hashtags, tweet_max_chars)
            fname = f"tweet_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Article":
            social = tpl_article(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"article_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "Instagram":
            social = tpl_instagram(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"instagram_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Facebook":
            social = tpl_facebook(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"facebook_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "YouTube Description":
            social = tpl_youtube_desc(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"youtube_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        elif platform == "Threads":
            social = tpl_threads(strategy, product_brand, target_audience, objective, hashtags)
            fname = f"threads_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        else:  # Medium Article
            social = tpl_medium(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"medium_{re.sub(r'\\W+','_',product_brand.lower())}.md"

        # Save to file in Colab for download
        out_path = Path(fname).resolve()
        out_path.write_text(social, encoding="utf-8")
        return strategy, social, str(out_path)

    # If LLM path (single name based on platform)
    map_name = {
        "LinkedIn": "linkedin",
        "X (Twitter)": "tweet",
        "Article": "article",
        "Instagram": "instagram",
        "Facebook": "facebook",
        "YouTube Description": "youtube",
        "Threads": "threads",
        "Medium Article": "medium",
    }
    fname = f"{map_name.get(platform,'post')}_{re.sub(r'\\W+','_',product_brand.lower())}.md"
    out_path = Path(fname).resolve()
    out_path.write_text(social, encoding="utf-8")
    return strategy, social, str(out_path)

# ----------------------------
# Build the simple UI (good for Colab)
# ----------------------------
with gr.Blocks(title="Marketing Crew → Social Content") as demo:
    gr.Markdown("## Marketing Strategy → Social Content")

    with gr.Row():
        brand = gr.Textbox(label="Product/Brand", placeholder="e.g., DDS AI Residency")
        audience = gr.Textbox(label="Target Audience", placeholder="e.g., Data science beginners in MENA")
        objective = gr.Textbox(label="Objective", placeholder="e.g., Drive applications for Cohort 8")

    with gr.Row():
        platform = gr.Dropdown(choices=PLATFORMS, value="LinkedIn", label="Platform")
        tone = gr.Dropdown(choices=["Professional", "Friendly", "Bold", "Educational", "Conversational"],
                           value="Professional", label="Tone")
        hashtags = gr.Textbox(label="Hashtags (comma separated)", placeholder="ai, generativeai, datascience")

    with gr.Row():
        use_llm = gr.Checkbox(value=False, label="Use LLM Copywriter")
        li_max_words = gr.Slider(100, 350, value=180, step=10, label="LinkedIn max words")
        tweet_max_chars = gr.Slider(120, 280, value=270, step=5, label="X/Tweet max chars")
        article_max_words = gr.Slider(400, 1200, value=800, step=50, label="Article/Medium max words")

    run_btn = gr.Button("Generate")

    gr.Markdown("### Strategy Output")
    strategy_md = gr.Markdown()

    gr.Markdown("### Platform Copy")
    social_md = gr.Markdown()

    download_file = gr.File(label="Download", interactive=False)

    run_btn.click(
        fn=generate,
        inputs=[brand, audience, objective, platform, tone, hashtags, use_llm,
                li_max_words, tweet_max_chars, article_max_words],
        outputs=[strategy_md, social_md, download_file]
    )

# In Colab, share=True is handy for preview links
demo.launch(share=True)


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [5]:
from crewai import Agent, Task, Crew
import gradio as gr
import re
from datetime import datetime
from pathlib import Path

# --- Logo: convert GitHub page URL to raw if needed ---
def to_raw_github(url: str) -> str:
    # Accepts both blob and raw URLs; converts blob → raw
    return url.replace("https://github.com/", "https://raw.githubusercontent.com/").replace("/blob/", "/")

LOGO_URL = to_raw_github("https://github.com/Decoding-Data-Science/airesidency/blob/main/dds_logo.jpg")

# ----------------------------
# Your existing agents (unchanged)
# ----------------------------
lead_market_analyst = Agent(
    role="Lead Market Analyst",
    goal="Deliver sharp, data-driven market insights for the brand/product.",
    backstory=("Senior analyst skilled in competitor intelligence, audience segmentation, "
               "channel dynamics, and market sizing. Outputs actionable insights."),
    allow_delegation=False,
    verbose=True,
)

chief_marketing_strategist = Agent(
    role="Chief Marketing Strategist",
    goal="Turn research into a focused, measurable go-to-market strategy.",
    backstory=("Veteran strategist who crafts positioning, messaging pillars, channel mix, "
               "and KPI frameworks; coordinates the team."),
    allow_delegation=False,
    verbose=True,
)

creative_content_creator = Agent(
    role="Creative Content Creator",
    goal="Transform the strategy into compelling campaign concepts and a content calendar.",
    backstory=("Concept-to-copy creative converting strategy into campaign ideas, ad copy, "
               "social posts, and long-form content."),
    allow_delegation=False,
    verbose=True,
)

# Optional: focused social copywriter (only if toggled)
social_copywriter = Agent(
    role="Social Copywriter",
    goal="Turn strategy into platform-appropriate copy with strong hooks and clear CTAs.",
    backstory=("Skilled at LinkedIn thought-leadership, X brevity, and long-form posts that convert."),
    allow_delegation=False,
    verbose=False,
)

# ----------------------------
# Core crew (unchanged)
# ----------------------------
def run_marketing_crew(product_brand: str, target_audience: str, objective: str) -> str:
    topic = f"{product_brand} | Audience: {target_audience} | Objective: {objective}"

    market_analysis_task = Task(
        description=(
            f"Conduct a concise market analysis for: {topic}. "
            "Cover: (1) ICP & segments, (2) JTBD/pain points & objections, "
            "(3) competitive landscape & whitespace, (4) demand signals & seasonality, "
            "(5) channel dynamics (search/social/email/partners/events), "
            "(6) keyword themes & content gaps, (7) risks/assumptions."
        ),
        expected_output=(
            "A structured brief with bullet points for each section and a 5–8 point summary of "
            "the most actionable insights."
        ),
        agent=lead_market_analyst
    )

    strategy_task = Task(
        description=(
            f"Using the Market Analysis brief, craft a go-to-market strategy for: {topic}. "
            "Include: positioning statement, value prop, 3–5 messaging pillars, "
            "priority segments, channel mix with rationale, offer/CTA ideas, "
            "90-day roadmap (phases & owners), KPI tree (primary/leading indicators), "
            "and a lightweight budget allocation (% by channel)."
        ),
        expected_output="A strategy document with the sections above plus a one-page executive summary.",
        agent=chief_marketing_strategist,
        context=[market_analysis_task]
    )

    creative_task = Task(
        description=(
            "Based on the Strategy, produce: (a) 3 campaign concepts (hook, angle, proof), "
            "(b) ad copy variants (paid search & paid social), "
            "(c) a 4-week content calendar (blog/LinkedIn/X/YouTube/Newsletter) with titles, "
            "briefs, CTAs, intended KPIs, and (d) a landing-page wireframe outline "
            "(hero, value blocks, social proof, FAQ)."
        ),
        expected_output="Campaign concepts + copy, a tabular content calendar, and a structured LP outline.",
        agent=creative_content_creator,
        context=[strategy_task]
    )

    crew = Crew(
        agents=[lead_market_analyst, chief_marketing_strategist, creative_content_creator],
        tasks=[market_analysis_task, strategy_task, creative_task],
        verbose=True
    )
    return crew.kickoff()

# ----------------------------
# Helpers
# ----------------------------
def _first_n_points(text: str, n: int = 5):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    bullets = []
    for l in lines:
        if l.startswith(("-", "*", "•")) or re.match(r"^\d+[\.\)]", l) or len(l) > 50:
            bullets.append(l.lstrip("-*• ").strip())
        if len(bullets) >= n: break
    if not bullets:
        parts = [p.strip() for p in text.split("\n\n") if p.strip()]
        bullets = parts[:n]
    return bullets[:n]

def _hashtags(csv_tags: str) -> str:
    tags = [t.strip().replace("#", "") for t in (csv_tags or "").split(",") if t.strip()]
    return "" if not tags else " " + " ".join(f"#{t}" for t in tags)

def _truncate_chars(s: str, max_chars: int) -> str:
    return s if len(s) <= max_chars else s[:max_chars - 1] + "…"

# ----------------------------
# Lightweight templates (no extra LLM call)
# ----------------------------
def tpl_linkedin(strategy, brand, audience, objective, hashtags, max_words=180):
    hook = f"{brand}: a sharper path to {objective} for {audience}."
    pts = "\n".join([f"- {p}" for p in _first_n_points(strategy, 5)])
    body = f"""**{hook}**

**Key insights**
{pts}

**Next 90 days**
- Prove the positioning with fast tests
- Double down on channels with strongest signals
- Track leading KPIs weekly

CTA: Comment “PLAYBOOK” if you want the GTM outline.{_hashtags(hashtags)}
"""
    words = body.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else body

def tpl_tweet(strategy, brand, audience, objective, hashtags, max_chars=270):
    points = _first_n_points(strategy, 3)
    msg = f"{brand} → {objective} for {audience}: " + " | ".join(_truncate_chars(p, 80) for p in points)
    return _truncate_chars(msg + _hashtags(hashtags), max_chars)

def tpl_article(strategy, brand, audience, objective, hashtags, max_words=800):
    intro = (f"{brand} is targeting {audience} to achieve {objective}. "
             "Here’s the distilled market analysis, GTM strategy, and a 90-day plan.")
    text = f"""# {brand}: GTM Playbook for {audience}

## Why this matters
{intro}

## Strategy in brief
- Positioning & messaging pillars
- Priority segments & channel mix
- Offers/CTAs, KPI tree
- 90-day roadmap & budget split

## Research & Strategy Notes
{strategy}

## What to do next
1) Validate 1–2 offers with tight ICP cohorts
2) Launch a 2-channel test with weekly KPI reviews
3) Scale what converts, archive what doesn’t

*Updated: {datetime.utcnow().strftime("%Y-%m-%d")}*{_hashtags(hashtags)}
"""
    words = text.split()
    return (" ".join(words[:max_words]) + "…") if len(words) > max_words else text

# ----------------------------
# Optional LLM copywriter (reuses Social Copywriter agent)
# ----------------------------
def llm_copywriter(strategy_text, brand, audience, objective, tone, platform,
                   hashtags, li_words, tweet_chars, article_words):
    limits = {
        "LinkedIn": f"≈ {li_words} words",
        "X (Twitter)": f"≤ {tweet_chars} chars",
        "Article": f"≈ {article_words} words",
    }
    req = f"""Create a {platform} post from the GTM strategy.

Brand: {brand}
Audience: {audience}
Objective: {objective}
Tone: {tone}
Hashtags: {hashtags or '(none)'}
Length limit: {limits.get(platform, 'keep concise')}

Requirements:
- Strong first-line hook
- 1–3 concrete insights from the strategy (no clichés)
- Clear CTA
- Respect platform style and length

--- STRATEGY ---
{strategy_text}
"""
    task = Task(description=req, expected_output=f"{platform} copy ready to publish.", agent=social_copywriter)
    crew = Crew(agents=[social_copywriter], tasks=[task], verbose=False)
    return crew.kickoff()

# ----------------------------
# Generation wrapper
# ----------------------------
PLATFORMS = ["LinkedIn", "X (Twitter)", "Article"]  # Reduced for better UX

def generate(product_brand, target_audience, objective,
             platform, tone, hashtags, use_llm,
             li_max_words, tweet_max_chars, article_max_words):

    if not product_brand or not target_audience or not objective:
        return "Please fill Brand, Audience, and Objective.", "", None

    # 1) Run the main Crew once
    strategy = run_marketing_crew(product_brand, target_audience, objective)

    # 2) Copy route
    if use_llm:
        social = llm_copywriter(strategy, product_brand, target_audience, objective,
                                tone, platform, hashtags,
                                li_max_words, tweet_max_chars, article_max_words)
    else:
        if platform == "LinkedIn":
            social = tpl_linkedin(strategy, product_brand, target_audience, objective, hashtags, li_max_words)
            fname = f"linkedin_{re.sub(r'\\W+','_',product_brand.lower())}.md"
        elif platform == "X (Twitter)":
            social = tpl_tweet(strategy, product_brand, target_audience, objective, hashtags, tweet_max_chars)
            fname = f"tweet_{re.sub(r'\\W+','_',product_brand.lower())}.txt"
        else:  # Article
            social = tpl_article(strategy, product_brand, target_audience, objective, hashtags, article_max_words)
            fname = f"article_{re.sub(r'\\W+','_',product_brand.lower())}.md"

        out_path = Path(fname).resolve()
        out_path.write_text(social, encoding="utf-8")
        return strategy, social, str(out_path)

    # Save LLM result with reasonable extension
    name_map = {"LinkedIn": "linkedin", "X (Twitter)": "tweet", "Article": "article"}
    fname = f"{name_map.get(platform,'post')}_{re.sub(r'\\W+','_',product_brand.lower())}.md"
    out_path = Path(fname).resolve()
    out_path.write_text(social, encoding="utf-8")
    return strategy, social, str(out_path)

# ----------------------------
# Theming & Layout (2 columns with header)
# ----------------------------
theme = gr.themes.Soft(primary_hue="indigo", neutral_hue="slate")

CUSTOM_CSS = """
#header {display:flex; align-items:center; gap:16px; padding:10px 14px; border-radius:12px;
         background: linear-gradient(90deg, #eef2ff, #f8fafc); border:1px solid #e5e7eb;}
#header img {width:44px; height:44px; object-fit:contain; border-radius:8px;}
#header .title {font-weight:700; font-size:18px; color:#111827;}
#header .subtitle {font-size:13px; color:#6b7280;}
.card {border:1px solid #e5e7eb; border-radius:12px; padding:12px; background:#ffffff;}
"""

with gr.Blocks(title="DDS Marketing Crew → Social Content", theme=theme, css=CUSTOM_CSS) as demo:
    with gr.Row():
        with gr.Column(scale=12):
            with gr.Row(elem_id="header"):
                gr.Image(value=LOGO_URL, show_label=False, interactive=False, height=48, width=48)
                gr.HTML("""
                    <div>
                      <div class="title">Decoding Data Science — Marketing Strategy → Social Generator</div>
                      <div class="subtitle">Run the Analyst → Strategist → Creator pipeline, then produce a platform-ready post.</div>
                    </div>
                """)

    with gr.Row():
        # Left: Inputs
        with gr.Column(scale=5):
            with gr.Group(elem_classes="card"):
                brand = gr.Textbox(label="Product/Brand", placeholder="e.g., DDS AI Residency", autofocus=True)
                audience = gr.Textbox(label="Target Audience", placeholder="e.g., Data science beginners in MENA")
                objective = gr.Textbox(label="Objective", placeholder="e.g., Drive applications for Cohort 8")

            with gr.Group(elem_classes="card"):
                platform = gr.Dropdown(choices=PLATFORMS, value="LinkedIn", label="Platform")
                tone = gr.Dropdown(choices=["Professional", "Friendly", "Bold", "Educational", "Conversational"],
                                   value="Professional", label="Tone")
                hashtags = gr.Textbox(label="Hashtags (comma separated)", placeholder="ai, generativeai, datascience")
                use_llm = gr.Checkbox(value=False, label="Use LLM Copywriter (higher fidelity)")

            with gr.Group(elem_classes="card"):
                li_max_words = gr.Slider(100, 350, value=180, step=10, label="LinkedIn max words")
                tweet_max_chars = gr.Slider(120, 280, value=270, step=5, label="X/Tweet max chars")
                article_max_words = gr.Slider(400, 1200, value=800, step=50, label="Article max words")

            run_btn = gr.Button("Generate Strategy → Post", variant="primary")

        # Right: Outputs
        with gr.Column(scale=7):
            with gr.Accordion("Strategy Output (from Crew)", open=True):
                strategy_md = gr.Markdown(value="*(Will appear here after generation)*")

            with gr.Group(elem_classes="card"):
                gr.Markdown("**Platform Copy** (editable; use the copy icon)")
                social_tb = gr.Textbox(lines=18, show_copy_button=True, label=None)

            download_file = gr.File(label="Download", interactive=False)

    run_btn.click(
        fn=generate,
        inputs=[brand, audience, objective, platform, tone, hashtags, use_llm,
                li_max_words, tweet_max_chars, article_max_words],
        outputs=[strategy_md, social_tb, download_file]
    )

# In Colab, share=True is handy
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12555cc06b6061f5fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
